In [ ]:
import os
import json
import time
import random
from tqdm import tqdm
import textwrap
import requests
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.patches as patches
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#project_id = 5795
project_id = 5654

# Data downloading and preparation

In [ ]:
url = 'https://tasks.hotosm.org/api/v1/project/' + str(project_id)
headers = {'Accept': 'application/json', 'Authorization': 'Token sessionTokenHere==', 'Accept-Language': 'en'}
r = requests.get(url, headers=headers)
%time data_project = r.json()

### List task ids and store them

In [ ]:
tasks_ids = list()
for feature in data_project['tasks']['features']:
    tasks_ids.append(feature['properties']['taskId'])
data_project['tasks_ids'] = tasks_ids

In [ ]:
tasks_ids[:10]

### Add some summary data

In [ ]:
url = 'https://tasks.hotosm.org/api/v1/project/' + str(project_id) + '/summary'    
r = requests.get(url, headers=headers)
summary = r.json()

In [ ]:
summary

In [ ]:
for key in summary:
    if key not in data_project.keys():
        data_project[key] = summary[key]

### Add task history data

In [ ]:
task_history = dict()
missing_taks = list()
for task_id in tqdm(tasks_ids):
    url = 'https://tasks.hotosm.org/api/v1/project/' + str(project_id) + '/task/' + str(task_id)    
    r = requests.get(url, headers=headers)
    if r.ok :
        task_history[task_id] = r.json()
    else :
        missing_taks.append(task_id)
    time.sleep(0.5 + random.random())
print(f'{len(missing_taks)} missing tasks')

In [ ]:
data_project['task_history'] = task_history

### Store data

In [ ]:
with open(os.path.join('..', 'data', str(project_id) + '.json'), 'w') as outfile: 
    json.dump(data_project, outfile)

# Mapping

### Load data

In [ ]:
with open(os.path.join('..', 'data', str(project_id) + '.json')) as f:
    data_project = json.load(f)

In [ ]:
priority_area =  data_project['priorityAreas'][0]

In [ ]:
start = pd.to_datetime(data_project['created']).date()

In [ ]:
nb_days = 0
task_history = data_project['task_history']
for task_id in data_project['tasks_ids']:
    date = pd.to_datetime(task_history[str(task_id)]['taskHistory'][0]['actionDate']).date()
    day = (date - start).days
    nb_days = max(nb_days, day)
print(f'{nb_days} days')

In [ ]:
locked_tasks = []
for i in range(nb_days + 1):
    locked_tasks.append(set())
locked_tasks[0:10]

In [ ]:
tasks_states = dict()

### Tasks data

In [ ]:
def get_task_states(task_data, locked_tasks, start, nb_days):
    """
    Processed each history of a task to fill locked_tasks and return states.
    
    Return a numpy array indexed by days from start with the following values
    * 0 : NOTHING
    * 1 : MAPPED
    * 2 : INVALIDATED
    * 3 : VALIDATED
    """
    task_states = np.zeros(nb_days + 1)
    for task_hist in reversed(task_data['taskHistory']):
        date = pd.to_datetime(task_hist['actionDate']).date()
        day = (date - start).days
        if task_hist['action'].startswith('LOCK'):
            locked_tasks[day].add(task_data['taskId'])
            continue
        if task_hist['action'] != 'STATE_CHANGE':
            continue
        if task_hist['actionText'] == 'MAPPED':
            task_states[day:] = 1
            continue
        if task_hist['actionText'] == 'INVALIDATED':
            task_states[day:] = 2
            continue
        if task_hist['actionText'] == 'VALIDATED':
            task_states[day:] = 3
            continue
        if task_hist['actionText'] == 'BADIMAGERY':
            task_states[day:] = 4
            continue
    return task_states

In [ ]:
task_history = data_project['task_history']
for task_id in data_project['tasks_ids']:
    tasks_states[task_id] = get_task_states(task_history[str(task_id)], locked_tasks, start, nb_days)

In [ ]:
tasks_states[data_project['tasks_ids'][0]]

### Plotting

In [ ]:
cartong_logo = Image.open(os.path.join('..', 'data', 'CartONG_logo.png'))

In [ ]:
legend = Image.open(os.path.join('..', 'data', 'Legend.png'))

In [ ]:
data_dir = os.path.join('..', 'data', str(project_id))

In [ ]:
os.makedirs(data_dir, exist_ok=True)

In [ ]:
for day in tqdm(range(nb_days + 1)):
    for plot_lock in [True, False]:
        fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 10), dpi=100, sharex=True)

        for feature in data_project['tasks']['features']:
            # Plot borders
            arr = np.array(feature['geometry']['coordinates'][0][0]).transpose()
            plt.plot(arr[0], arr[1], color='black')

            # Plot locking or state
            task_id = feature['properties']['taskId']
            state = tasks_states[task_id][day]
            if plot_lock and task_id in locked_tasks[day]:
                ax.add_patch(patches.Polygon(arr.transpose(), color=(159/255., 188/255., 247/255.))) # blue
                continue
            if state == 1:
                ax.add_patch(patches.Polygon(arr.transpose(), color=(254/255., 231/255., 156/255.))) # yellow
                continue
            if state == 2:
                ax.add_patch(patches.Polygon(arr.transpose(), color=(245/255., 156/255., 178/255.))) # pink
                continue
            if state == 3:
                ax.add_patch(patches.Polygon(arr.transpose(), color=(152/255., 203/255., 151/255.))) # green
                continue
            if state == 4:
                ax.add_patch(patches.Polygon(arr.transpose(), color=(152/255., 152/255., 151/255.))) # black 
                continue

        # Plot priority area
        for priority_area in data_project['priorityAreas']:
            ax.add_patch(patches.Polygon(priority_area['coordinates'][0], fill=False, color='r', lw=2))

        str_day_title = (start + pd.Timedelta(days=day)).strftime('%d-%m-%Y')
        title = '\n'.join(textwrap.wrap(data_project['name'] + ' #' + str(project_id), 50)) + '\n'+ str_day_title
        ax.set_title(title, fontsize=16)
        ax.axis('off')

        # Save plot
        str_day_file = (start + pd.Timedelta(days=day)).strftime('%Y-%m-%d')
        suffix = '_2' if not plot_lock else ''
        file_path = os.path.join(data_dir, str_day_file + suffix + '.png')
        plt.savefig(file_path, dpi=100)
        plt.close()

        # Add CartONG logo
        im = Image.open(file_path)
        im.paste(cartong_logo, (0,0))
        im.paste(legend, (0, 1000-legend.size[1]))
        im.save(file_path)